In [49]:
import cv2 # open cv library
import cvzone
import time # 
import os # 
import HandTrackingModule as h_mod
import mediapipe as mp
import random

In [50]:
cap = cv2.VideoCapture(1)
# set video frame width to 640px
cap.set(3, 640)
# set video frame height to 640px
cap.set(4, 480)
# initialize Hand Detector
detector = h_mod.HandDetector(maxHands=1)
# initialize timer
timer = 0
# initialize state
stateResult = False
# initialize flag
startGame = False
# initialize Score
scores = [0,0] # [AI Score, Player Score]
# initialize Fair Mode (random AI Move)
fairMode = True


In [51]:
while True:
    # background img
    imgBG = cv2.imread("Resources/BG2.png")

    success, img = cap.read()

    # resize img to fit in Background box
    imgScaled = cv2.resize(img, (0,0), None, 0.875, 0.875)
    imgScaled = imgScaled[:,80:480]
    # flip image
    imgScaled = cv2.flip(imgScaled, 1)

    # Find Hands
    hands, img = detector.findHands(imgScaled)
    # Game
    if startGame:
        if stateResult is False:
            timer = time.time() - initialTime
            cv2.putText(imgBG, str(int(timer)), (610, 435), cv2.FONT_HERSHEY_PLAIN, 5, (255,0,0), 5)
            
            if timer>3:
                stateResult = True
                timer = 0

                if hands:
                    hand = hands[0]
                    playerMove = None
                    fingers = detector.fingersUp(hand)
                    # Check Player moves
                    if fingers == [0,0,0,0,0] or fingers == [1,0,0,0,0]:
                        playerMove = 1 # Rock
                    if fingers == [1,1,1,1,1] or fingers == [0,1,1,1,1]:
                        playerMove = 2 # Paper
                    if fingers == [0,1,1,0,0] or fingers == [1,1,1,0,0]:
                        playerMove = 3 # Scissors
                    
                    # Generate AI Move
                    if fairMode:
                        AIMove = random.randint(1,3) # random Int (1,2,3)
                    else:
                        if playerMove==1:
                            AIMove = 2
                        elif playerMove==2:
                            AIMove = 3
                        else:
                            AIMove = 1
                    moves = [playerMove, AIMove]
                    imgAI = cv2.imread(f'Resources/{AIMove}.png', cv2.IMREAD_UNCHANGED)
                    imgBG = cvzone.overlayPNG(imgBG, imgAI, (150, 310))
                    
                    # Check if Player or AI won
                    # Player wins
                    if moves==[1,3] or moves==[2,1] or moves==[3,2]:
                        scores[1]+=1
                    # AI wins
                    if moves==[1,2] or moves==[2,3] or moves==[3,1]:
                        scores[0]+=1
                    # print(moves)
                    # print(scores)


    # insert Scaled image in Background
    imgBG[234:654, 795:1195] = imgScaled

    # show AI Move permanently
    if stateResult:
        imgBG = cvzone.overlayPNG(imgBG, imgAI, (150, 310))
    
    # show scores
    cv2.putText(imgBG, str(scores[0]), (410, 215), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255), 5) # AI Score
    cv2.putText(imgBG, str(scores[1]), (1115, 215), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255), 5) # Player Score

    cv2.imshow("Image", img)
    cv2.imshow("BG", imgBG)
    cv2.imshow("BG", imgScaled)
    key = cv2.waitKey(1)
    # Press S to start game
    if key == ord('s'):
        startGame = True
        initialTime = time.time()
        stateResult = False

    # Press Q to quit loop and destroy windows
    if key == ord('q'):
        cv2.destroyAllWindows()
        for i in range(2):
            cv2.waitKey(1)
        break
